# Exploração da base de dados do INEP

Neste notebook, iremos carregar e realizar exploração de dados.

In [1]:
#DEFININDO DIRETÓRIO DE DADOS
dir = '/Users/john-/OneDrive - Universidade Federal da Paraíba/Área de Trabalho/Artigo - Estatística/projeto-evasao/data/raw'

#IMPORTANDO BIBLIOTECAS
import pandas as pd
import re
import os

In [2]:
#LENDO DADOS:

#LENDO DADOS - INDICADORES_2018_2023

arq = f'{dir}/indicadores/indicadores_2018_2023.xlsx'
indicadores_2018_2023 = pd.read_excel(arq, header=6)
indicadores_2018_2023.columns = indicadores_2018_2023.iloc[1]
indicadores_2018_2023 = indicadores_2018_2023.iloc[2:].reset_index(drop=True)

#LENDO DADOS - CPC 2018 - 2023
anos = [ano for ano in range(2018, 2024) if ano != 2020]
for ano in anos:
    arq = f'{dir}/cpc/cpc_{ano}.xlsx'
    df = pd.read_excel(arq, header=0)
    df.columns = [str(col).strip().replace(' ', '_').upper() for col in df.columns]
    if 'CO_UF' in df.columns:
        df['CO_UF'] = df['CO_UF'].fillna(0).astype(int)
    exec(f"cpc_{ano} = df")

#LENDO DADOS - ENADE 2018 - 2023
anos = [ano for ano in range(2018, 2024) if ano != 2020]
for ano in anos:
    arq = f'{dir}/enade/enade_{ano}.xlsx'
    df = pd.read_excel(arq, header=0)
    df.columns = [str(col).strip().replace(' ', '_').upper() for col in df.columns]
    exec(f"enade_{ano} = df")

#LENDO DADOS - ENADE 2018 - 2023
anos = [ano for ano in range(2018, 2024) if ano != 2020]
for ano in anos:
    arq = f'{dir}/idd/idd_{ano}.xlsx'
    df = pd.read_excel(arq, header=0)
    df.columns = [str(col).strip().replace(' ', '_').upper() for col in df.columns]
    exec(f"idd_{ano} = df")

#LENDO DADOS - IGC 2018 - 2023
anos = [ano for ano in range(2018, 2024) if ano != 2020]
for ano in anos:
    arq = f'{dir}/igc/igc_{ano}.xlsx'
    df = pd.read_excel(arq, header=0)
    df.columns = [str(col).strip().replace(' ', '_').upper() for col in df.columns]
    exec(f"igc_{ano} = df")



In [3]:
#FILTRANDO DADOS - VARIAVEIS INDEPENDENTES - CADECORIA ADMINISTRATIVA: UNIVERSIDADE FEDERAL

def filtrar_publica_federal(prefixos, anos, categoria='Pública Federal'):
    for prefixo in prefixos:
        for ano in anos:
            var_name = f"{prefixo}_{ano}"
            if var_name in globals():
                df = globals()[var_name]

                # Identifica a coluna da categoria administrativa (com ou sem '*')
                col_match = [col for col in df.columns if re.match(r'CATEGORIA_ADMINISTRATIVA\*?', col)]

                if col_match:
                    col_name = col_match[0]
                    new_col_name = re.sub(r'\*$', '', col_name)

                    if col_name != new_col_name:
                        df.rename(columns={col_name: new_col_name}, inplace=True)

                    print(f"{var_name}: Valores únicos em '{new_col_name}' antes de filtrar: {df[new_col_name].unique()}")

                    # Filtra e atualiza a variável original
                    df_filtered = df[df[new_col_name] == categoria].reset_index(drop=True)
                    globals()[var_name] = df_filtered

                    print(f"{var_name}: {len(df_filtered)} registros após filtrar para '{categoria}'.\n")
                else:
                    print(f"{var_name}: Coluna 'CATEGORIA_ADMINISTRATIVA' não encontrada.\n")
            else:
                print(f"Variável '{var_name}' não encontrada no ambiente.\n")

# ✅ Parâmetros
prefixos = ['cpc', 'enade', 'idd', 'igc']
anos = [2018, 2019, 2021, 2022, 2023]

# ✅ Execução
filtrar_publica_federal(prefixos, anos)

#FILTRANDO DADOS - VARIAVEL DEPENDENTE - CADECORIA ADMINISTRATIVA: UNIVERSIDADE FEDERAL

# Verifica se a variável existe no ambiente
if 'indicadores_2018_2023' in globals():
    df = indicadores_2018_2023
    
    # Verifica se a coluna existe
    if 'TP_CATEGORIA_ADMINISTRATIVA' in df.columns:
        print(f"Valores únicos antes do filtro: {df['TP_CATEGORIA_ADMINISTRATIVA'].unique()}")

        # Filtra os dados para TP_CATEGORIA_ADMINISTRATIVA == 1
        df_filtrado = df[df['TP_CATEGORIA_ADMINISTRATIVA'] == 1].reset_index(drop=True)
        
        # Atualiza a variável original
        indicadores_2018_2023 = df_filtrado

        print(f"Registros após o filtro: {len(indicadores_2018_2023)}")
    else:
        print("Coluna 'TP_CATEGORIA_ADMINISTRATIVA' não encontrada.")
else:
    print("Variável 'indicadores_2018_2023' não encontrada no ambiente.")


cpc_2018: Valores únicos em 'CATEGORIA_ADMINISTRATIVA' antes de filtrar: ['Pública Federal' 'Pública Estadual' 'Privada sem fins lucrativos'
 'Especial' 'Pública Municipal' 'Privada com fins lucrativos']
cpc_2018: 692 registros após filtrar para 'Pública Federal'.

cpc_2019: Valores únicos em 'CATEGORIA_ADMINISTRATIVA' antes de filtrar: ['Pública Federal' 'Pública Estadual' 'Privada sem fins lucrativos'
 'Pública Municipal' 'Privada com fins lucrativos' 'Especial']
cpc_2019: 1426 registros após filtrar para 'Pública Federal'.

cpc_2021: Valores únicos em 'CATEGORIA_ADMINISTRATIVA' antes de filtrar: ['Pública Federal' 'Pública Estadual' 'Privada sem fins lucrativos'
 'Especial' 'Pública Municipal' 'Privada com fins lucrativos' nan]
cpc_2021: 2245 registros após filtrar para 'Pública Federal'.

cpc_2022: Valores únicos em 'CATEGORIA_ADMINISTRATIVA' antes de filtrar: ['Pública Federal' 'Pública Estadual' 'Privada sem fins lucrativos'
 'Especial' 'Pública Municipal' 'Privada com fins lucra

In [4]:
#RENOMEANDO COLUNA CÓDIGO DO CURSO DA VARIAVEL INDICADORES 2018_2023
if 'indicadores_2018_2023' in globals():
    df = indicadores_2018_2023

    if 'CO_CURSO' in df.columns:
        df.rename(columns={'CO_CURSO': 'CÓDIGO_DO_CURSO'}, inplace=True)
        print("Coluna 'CO_CURSO' renomeada para 'CÓDIGO_DO_CURSO'.")
    else:
        print("A coluna 'CO_CURSO' não foi encontrada.")
else:
    print("A variável 'indicadores_2018_2023' não está disponível no ambiente.")


Coluna 'CO_CURSO' renomeada para 'CÓDIGO_DO_CURSO'.


In [5]:
#CRIANDO DATAFRAME - MÉDIAS DAS VARIAVEIS - CPC - 2018 - 2023

import pandas as pd

# Suponha que estas variáveis estejam carregadas
dfs_cpc = [cpc_2018, cpc_2019, cpc_2021, cpc_2022, cpc_2023]

colunas_fixas = [
    'CÓDIGO_DA_IES', 'NOME_DA_IES', 'SIGLA_DA_IES', 'ORGANIZAÇÃO_ACADÊMICA',
    'CATEGORIA_ADMINISTRATIVA', 'CÓDIGO_DO_CURSO', 'CÓDIGO_DA_ÁREA',
    'ÁREA_DE_AVALIAÇÃO', 'MODALIDADE_DE_ENSINO', 'CÓDIGO_DO_MUNICÍPIO',
    'MUNICÍPIO_DO_CURSO', 'SIGLA_DA_UF'
]

colunas_para_media = [
    'NOTA_PADRONIZADA_-_IDD',
    'NOTA_PADRONIZADA_-_ORGANIZAÇÃO_DIDÁTICO-PEDAGÓGICA',
    'NOTA_PADRONIZADA_-_INFRAESTRUTURA_E_INSTALAÇÕES_FÍSICAS',
    'NOTA_PADRONIZADA_-_OPORTUNIDADE_DE_AMPLIAÇÃO_DA_FORMAÇÃO',
    'NOTA_PADRONIZADA_-_MESTRES',
    'NOTA_PADRONIZADA_-_DOUTORES'
]

# Junta todos os anos
df_cpc_todos = pd.concat(dfs_cpc, ignore_index=True)

# Converte colunas de notas para numérico (substitui '-' e outros por NaN)
for coluna in colunas_para_media:
    df_cpc_todos[coluna] = pd.to_numeric(df_cpc_todos[coluna], errors='coerce')

# Calcula médias por curso
media_cpc_agrupado = df_cpc_todos.groupby('CÓDIGO_DO_CURSO')[colunas_para_media].mean().reset_index()

# Remove duplicata e junta dados fixos
colunas_fixas_sem_codigo = [col for col in colunas_fixas if col != 'CÓDIGO_DO_CURSO']
valores_fixos = df_cpc_todos.drop_duplicates(subset='CÓDIGO_DO_CURSO')[['CÓDIGO_DO_CURSO'] + colunas_fixas_sem_codigo]

# Merge final
media_cpc_2018_2023 = pd.merge(media_cpc_agrupado, valores_fixos, on='CÓDIGO_DO_CURSO', how='left')

# Organiza colunas
media_cpc_2018_2023 = media_cpc_2018_2023[colunas_fixas + colunas_para_media]

# Visualiza os dados
media_cpc_2018_2023.head()


,CÓDIGO_DA_IES,NOME_DA_IES,SIGLA_DA_IES,ORGANIZAÇÃO_ACADÊMICA,CATEGORIA_ADMINISTRATIVA,CÓDIGO_DO_CURSO,CÓDIGO_DA_ÁREA,ÁREA_DE_AVALIAÇÃO,MODALIDADE_DE_ENSINO,CÓDIGO_DO_MUNICÍPIO,MUNICÍPIO_DO_CURSO,SIGLA_DA_UF,NOTA_PADRONIZADA_-_IDD,NOTA_PADRONIZADA_-_ORGANIZAÇÃO_DIDÁTICO-PEDAGÓGICA,NOTA_PADRONIZADA_-_INFRAESTRUTURA_E_INSTALAÇÕES_FÍSICAS,NOTA_PADRONIZADA_-_OPORTUNIDADE_DE_AMPLIAÇÃO_DA_FORMAÇÃO,NOTA_PADRONIZADA_-_MESTRES,NOTA_PADRONIZADA_-_DOUTORES
0,1.0,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,Universidade,Pública Federal,1.0,2.0,DIREITO,Educação Presencial,5103403.0,CUIABA,MT,2.529753,1.423914,2.120565,3.073949,4.856420,4.304179
1,1.0,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,Universidade,Pública Federal,2.0,13.0,CIÊNCIAS ECONÔMICAS,Educação Presencial,5103403.0,CUIABA,MT,1.732122,2.013531,2.727243,2.513958,4.394841,4.226191
2,1.0,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,Universidade,Pública Federal,3.0,5710.0,ENGENHARIA CIVIL,Educação Presencial,5103403.0,Cuiabá,MT,3.080000,0.000000,0.009500,0.341000,3.527000,3.025000
3,1.0,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,Universidade,Pública Federal,7.0,38.0,SERVIÇO SOCIAL,Educação Presencial,5103403.0,CUIABA,MT,2.911143,2.454815,1.476143,2.266447,4.663439,4.106887
4,1.0,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,Universidade,Pública Federal,8.0,22.0,CIÊNCIAS CONTÁBEIS,Educação Presencial,5103403.0,CUIABA,MT,2.363577,2.387500,2.737781,2.754933,4.777875,3.289096


In [6]:
#CRIANDO DATAFRAME - MÉDIAS DAS VARIAVEIS - ENADE - 2018 - 2023

# Lista de dataframes de ENADE por ano
enade_dfs = [enade_2018, enade_2019, enade_2021, enade_2022, enade_2023]

# Seleciona só o código do curso e a coluna de conceito contínuo
enade_conceitos = pd.concat([
    df[['CÓDIGO_DO_CURSO', 'CONCEITO_ENADE_(CONTÍNUO)']] for df in enade_dfs
], ignore_index=True)

# Converte valores para numérico (caso haja erros ou '-')
enade_conceitos['CONCEITO_ENADE_(CONTÍNUO)'] = pd.to_numeric(enade_conceitos['CONCEITO_ENADE_(CONTÍNUO)'], errors='coerce')

# Calcula a média contínua por curso
media_enade = enade_conceitos.groupby('CÓDIGO_DO_CURSO')['CONCEITO_ENADE_(CONTÍNUO)'].mean().reset_index()

# Renomeia a coluna
media_enade.rename(columns={'CONCEITO_ENADE_(CONTÍNUO)': 'MEDIA_CONCEITO_ENADE_(CONTÍNUO)'}, inplace=True)

# Junta com o dataframe principal
media_cpc_2018_2023 = pd.merge(media_cpc_2018_2023, media_enade, on='CÓDIGO_DO_CURSO', how='left')


In [7]:
#CRIANDO DATAFRAME - MÉDIAS DAS VARIAVEIS - IDD - 2018 - 2023

# Lista de dataframes de IDD por ano
idd_dfs = [idd_2018, idd_2019, idd_2021, idd_2022, idd_2023]

# Seleciona apenas as colunas relevantes
idd_conceitos = pd.concat([
    df[['CÓDIGO_DO_CURSO', 'IDD_(CONTÍNUO)']] for df in idd_dfs
], ignore_index=True)

# Converte para numérico (caso haja valores como '-')
idd_conceitos['IDD_(CONTÍNUO)'] = pd.to_numeric(idd_conceitos['IDD_(CONTÍNUO)'], errors='coerce')

# Calcula a média por curso
media_idd = idd_conceitos.groupby('CÓDIGO_DO_CURSO')['IDD_(CONTÍNUO)'].mean().reset_index()

# Junta com o dataframe principal
media_cpc_2018_2023 = pd.merge(media_cpc_2018_2023, media_idd, on='CÓDIGO_DO_CURSO', how='left')


In [8]:
#CRIANDO DATAFRAME - MÉDIAS DAS VARIAVEIS - IGC - 2018 - 2023

# Corrige o nome da coluna no igc_2021
igc_2021.rename(columns={'CÓDIGO_DA_IES*': 'CÓDIGO_DA_IES'}, inplace=True)

# Lista de DataFrames IGC
igc_dfs = [igc_2018, igc_2019, igc_2021, igc_2022, igc_2023]

# Concatena e calcula a média
igc_conceitos = pd.concat([
    df[['CÓDIGO_DA_IES', 'CONCEITO_MÉDIO_DE_GRADUAÇÃO', 'IGC_(CONTÍNUO)']] for df in igc_dfs
], ignore_index=True)

igc_conceitos['CONCEITO_MÉDIO_DE_GRADUAÇÃO'] = pd.to_numeric(igc_conceitos['CONCEITO_MÉDIO_DE_GRADUAÇÃO'], errors='coerce')
igc_conceitos['IGC_(CONTÍNUO)'] = pd.to_numeric(igc_conceitos['IGC_(CONTÍNUO)'], errors='coerce')

# Agrupamento por IES e média
igc_media = igc_conceitos.groupby('CÓDIGO_DA_IES').mean().reset_index()

# Junta ao DataFrame principal
media_cpc_2018_2023 = media_cpc_2018_2023.merge(igc_media, on='CÓDIGO_DA_IES', how='left')



In [9]:
#CRIANDO DATAFRAME - CÓPIANDO - TAXA DE DESISTÊNCIA CUMULADA - 2018 - 2023

# Remove linhas com valores ausentes em CÓDIGO_DO_CURSO
media_cpc_2018_2023 = media_cpc_2018_2023[media_cpc_2018_2023['CÓDIGO_DO_CURSO'].notna()]
indicadores_2018_2023 = indicadores_2018_2023[indicadores_2018_2023['CÓDIGO_DO_CURSO'].notna()]

# Converte as colunas CÓDIGO_DO_CURSO para inteiro
media_cpc_2018_2023['CÓDIGO_DO_CURSO'] = media_cpc_2018_2023['CÓDIGO_DO_CURSO'].astype(int)
indicadores_2018_2023['CÓDIGO_DO_CURSO'] = indicadores_2018_2023['CÓDIGO_DO_CURSO'].astype(int)

# Cria novo DataFrame apenas com CÓDIGO_DO_CURSO e TDA
tda_df = indicadores_2018_2023[['CÓDIGO_DO_CURSO', 'TDA']].drop_duplicates(subset='CÓDIGO_DO_CURSO')

# Merge com a base principal
media_cpc_2018_2023 = media_cpc_2018_2023.merge(tda_df, on='CÓDIGO_DO_CURSO', how='left')

ml_iq_2018_2023 = media_cpc_2018_2023


In [10]:
#DROPANDO MISSING VALUE

linhas_antes = len(ml_iq_2018_2023)
ml_iq_2018_2023 = ml_iq_2018_2023.dropna()
linhas_depois = len(ml_iq_2018_2023)
print(f"{linhas_antes - linhas_depois} linhas com valores ausentes foram removidas.")



164 linhas com valores ausentes foram removidas.


In [11]:
import os

# Crie uma cópia explícita, se necessário (supondo que ml_iq_2018_2023 seja uma fatia de outro DataFrame)
ml_iq_2018_2023 = ml_iq_2018_2023.copy()

# Calcular a mediana
mediana = ml_iq_2018_2023['TDA'].median()

# Criar a coluna binária de forma segura
ml_iq_2018_2023.loc[:, 'TDA_binaria'] = (ml_iq_2018_2023['TDA'] > mediana).astype(int)

# Caminho completo (pasta + nome do arquivo)
caminho_arquivo = os.path.join("..", "data", "processed", "ml_iq_2018_2023.csv")

# Salvar DataFrame
ml_iq_2018_2023.to_csv(caminho_arquivo, index=False)

print(f'DataFrame salvo em: {caminho_arquivo}')




DataFrame salvo em: ..\data\processed\ml_iq_2018_2023.csv
